In [7]:
! pip install bayes_opt

ERROR: Could not find a version that satisfies the requirement bayes_opt (from versions: none)
ERROR: No matching distribution found for bayes_opt
You should consider upgrading via the 'c:\users\patri\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [9]:
from warnings import simplefilter, filterwarnings
from sklearn.exceptions import ConvergenceWarning
import warnings
#from bayes_opt import BayesianOptimization

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

simplefilter(action='ignore', category=FutureWarning)
filterwarnings('ignore')

# Menu
<a name="navegacao"></a>

## 1) [Data cleaning and feature engineering](#cleaning)
- 1.1 [Encoding de Conglomerado e Natureza Jurídica](#encoding)
- 1.2 [Descrição dos dados](#descricao)
- 1.3 [Feature engineering](#engineering)
- 1.4 [Train/Test partition](#partition)


## 2) [Baysian Optimization for Hyperparameters Tunning](#bayesian)
- 2.1 [Suport Vector Machine](#svm0)
- 2.2 [Stochastic Gradient Descent](#sgd0)
- 2.3 [Multi-Task Elastic-Net](#elastic0)
- 2.4 [Least Angle Regression Model (LAR)](#LAR0)
- 2.5 [Ridge Regression](#ridge0)
- 2.6 [ElasticNet Regression](#ElasNet0)
- 2.7 [Bayesian Ridge](#bayesian0)
- 2.8 [Lasso Regression](#lasso0)
- 2.9 [Multilayer Perceptron Regression](#mlp0)
- 2.10 [Nearest Neighbors Regression](#nnr0)
- 2.11 [Resumo dos resultados](#resumo_rs0)


## 3) [Random Search for Hyperparameters Tunning](#randomsearch)
- 3.1 [Suport Vector Machine](#svm1)
- 3.2 [Stochastic Gradient Descent](#sgd1)
- 3.3 [Multi-Task Elastic-Net](#elastic1)
- 3.4 [Least Angle Regression Model (LAR)](#LAR1)
- 3.5 [Ridge Regression](#ridge1)
- 3.6 [ElasticNet Regression](#ElasNet1)
- 3.7 [Bayesian Ridge](#bayesian1)
- 3.8 [Lasso Regression](#lasso1)
- 3.9 [Multilayer Perceptron Regression](#mlp1)
- 3.10 [Nearest Neighbors Regression](#nnr1)
- 3.11 [Resumo dos resultados](#resumo_rs1)


## 4) [Random Halving Serach for Hyperparameters Tunning](#halving)
- 4.1 [Suport Vector Machine](#svm2)
- 4.2 [Stochastic Gradient Descent](#sgd2)
- 4.3 [Multi-Task Elastic-Net](#elastic2)
- 4.4 [Least Angle Regression Model (LAR)](#LAR2)
- 4.5 [Ridge Regression](#ridge2)
- 4.6 [ElasticNet Regression](#ElasNet2)
- 4.7 [Bayesian Ridge](#bayesian2)
- 4.8 [Lasso Regression](#lasso2)
- 4.9 [Multilayer Perceptron Regression](#mlp2)
- 4.10 [Nearest Neighbors Regression](#nnr2)
- 4.11 [Resumo dos resultados](#resumo_rs2)

In [10]:
from time import time
import requests
import pandas as pd
import numpy as np 
import random 
import os
import pickle
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [12]:
%%time

spread = pd.read_sas('dados bndes/dadospreadpy.sas7bdat' , encoding='latin-1')
#https://www.analyticsvidhya.com/blog/2018/03/introduction-regression-splines-python-codes/

Wall time: 1.25 s


### Veja tutorial sobre lags em um painel de dados:

- https://towardsdatascience.com/timeseries-data-munging-lagging-variables-that-are-distributed-across-multiple-groups-86e0a038460c

### Veja tutorial sobre Bayesian Hyperparameter Tunning for ML:

- https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f

In [13]:
#spread.columns

<a name="cleaning"></a>


# 1) Data Cleaning and Feature Engineering

- ir para [Menu Principal](#navegacao)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Train/Test partition](#partition)


<a name="encoding"></a>

## 1.1) Encoding de Conglomerado e Natureza Jurídica


In [14]:
%%time

spread.fillna(0,inplace=True)

Wall time: 27 ms


In [15]:
%%time

enc_congl = spread[['CNPJ','Conglomerado']].groupby(['Conglomerado']).count().reset_index().rename(columns={"CNPJ":"banco_enc"})
enc_congl.head()

Wall time: 52 ms


,Conglomerado,banco_enc
0,0,23462
1,ABC-BRASIL,289
2,ALFA,435
3,BANCOOB,389
4,BANESTES,34


In [16]:
%%time
spread = pd.merge( spread, enc_congl, left_on=['Conglomerado'],right_on=['Conglomerado'], how='inner')

Wall time: 120 ms


In [17]:
%%time

enc_natjur = spread[['CNPJ','NATJUR']].groupby(['NATJUR']).count().reset_index().rename(columns={"CNPJ":"firma_enc"})
enc_natjur.head()

Wall time: 59 ms


,NATJUR,firma_enc
0,1015,6
1,1031,65
2,1112,8
3,1244,2
4,2011,28


In [18]:
%%time
spread = pd.merge( spread, enc_natjur, left_on=['NATJUR'],right_on=['NATJUR'], how='inner')

Wall time: 114 ms


In [19]:
spread.loc[ (spread['TCB']=='b1'), 'banco_b1'] = 1
spread.loc[~(spread['TCB']=='b1'), 'banco_b1'] = 0

spread.loc[ (spread['TCB']=='b2'), 'banco_b2'] = 1
spread.loc[~(spread['TCB']=='b2'), 'banco_b2'] = 0

spread.loc[ (spread['TCB']=='b3C'), 'banco_b3'] = 1
spread.loc[~(spread['TCB']=='b3C'), 'banco_b3'] = 0

spread[['CNPJ','EMPRESA','banco_b1']].groupby(['banco_b1']).count()

,CNPJ,EMPRESA
banco_b1,,
0.0,24450,24450
1.0,34647,34647


In [20]:
spread[['CNPJ','EMPRESA','banco_b3']].groupby(['banco_b3']).count()

,CNPJ,EMPRESA
banco_b3,,
0.0,58868,58868
1.0,229,229


In [21]:

spread[['CNPJ','EMPRESA','TCB']].groupby(['TCB']).count()

,CNPJ,EMPRESA
TCB,,
b1,34647,34647
b2,22218,22218
b3C,229,229
b4,1447,1447
n1,556,556


<a name="engineering"></a>

## 1.2) Feature engineering
- ir para [Menu Principal](#navegacao)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Train/Test partition](#partition)


In [22]:
%%time
spread['target'] = spread['SpreadInstituicaoFinanceira'] 

# dados da operação
spread['oper_valor']                   = np.log(spread['ValorOperacao']) 
spread['oper_spreadbndes']             = spread['SpreadBNDES'] 
spread['oper_juros']                   = spread['Juros'] 
spread['oper_amortizacao']             = np.log(spread['Prazo_Amortiza']) 
spread['oper_carencia']                = np.log(spread['prazo_carencia']+1) 
spread['oper_valor_spreadbndes']       = np.log(spread['ValorOperacao']) - spread['SpreadBNDES']
spread['oper_valor_juros']             = np.log(spread['ValorOperacao']) - spread['Juros']
spread['oper_valor_amortizacao']       = np.log(spread['ValorOperacao']) - spread['Prazo_Amortiza']
spread['oper_valor_carencia']          = np.log(spread['ValorOperacao']) - spread['prazo_carencia'] 
spread['oper_spreadbndes_juros']       = spread['SpreadBNDES']           - spread['Juros'] 
spread['oper_spreadbndes_amortizacao'] = spread['SpreadBNDES']           - spread['Prazo_Amortiza']
spread['oper_spreadbndes_carencia']    = spread['SpreadBNDES']           - spread['prazo_carencia']
spread['oper_juros_amortizacao']       = spread['Juros']                 - spread['Prazo_Amortiza']
spread['oper_juros_carencia']          = spread['Juros']                 - spread['prazo_carencia']
spread['oper_amortizacao_carencia']    = spread['Prazo_Amortiza']        - spread['prazo_carencia']

Wall time: 23 ms


In [23]:
%%time
# dados da firma
spread['firma_potot']  = np.log(spread['POTOT_FIRMA']+1) 
spread['firma_idade']  = np.log(spread['IDADE_FIRMA']+1) 
spread['firma_massa']  = np.log(spread['MSAL_FIRMA']+1) 
spread['firma_po3gr']  = np.log(spread['PO3G_FIRMA']+1) 
spread['firma_rotat']  = np.log(spread['ROT_FIRMA']+1) 
spread['firma_engen']  = np.log(spread['ENG_FIRMA']+1) 
spread['firma_share']  = np.log(spread['SHARE_FIRMA']+1) 

spread['firma_potot_idade'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['IDADE_FIRMA']+2) 
spread['firma_potot_massa'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['MSAL_FIRMA']+2) 
spread['firma_potot_po3gr'] = np.log(spread['POTOT_FIRMA']+1) / np.log(spread['PO3G_FIRMA']+2) 
spread['firma_idade_massa'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['MSAL_FIRMA']+2) 
spread['firma_idade_po3gr'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['PO3G_FIRMA']+2) 
spread['firma_idade_engen'] = np.log(spread['IDADE_FIRMA']+1) / np.log(spread['ENG_FIRMA']+2)
spread['firma_massa_po3gr'] = np.log(spread['MSAL_FIRMA']+1)  / np.log(spread['PO3G_FIRMA']+2) 


Wall time: 33 ms


In [24]:
# dados da instituição financeira
spread['banco_ativo']            =np.log(spread['Ativo_Total']) 
spread['banco_carteira']         =np.log(spread['Carteira_Credito']) 
spread['banco_passivo']          =np.log(spread['Passivo_Circulante']) 
spread['banco_captacoes']        =np.log(spread['Captacoes']+1) 
spread['banco_patrimonio']       =np.log(spread['Patrimonio_Liquido']+1) 
spread['banco_lucroliquido']     =np.log(spread['Lucro_Liquido']+1) 
spread['banco_numagencias']      =np.log(spread['Numero_Agencias']+1) 
spread['banco_numeropostos']     =np.log(spread['Numero_Postos']+1) 
spread['banco_ativo_carteira']   =np.log(spread['Ativo_Total'])        /np.log(spread['Carteira_Credito'])   
spread['banco_ativo_passivo']    =np.log(spread['Ativo_Total'])        /np.log(spread['Passivo_Circulante']) 
spread['banco_ativo_captacoes']  =np.log(spread['Ativo_Total'])        /np.log(spread['Captacoes']+1)        
spread['banco_ativo_patrimonio'] =np.log(spread['Ativo_Total'])        /np.log(spread['Patrimonio_Liquido']) 
spread['banco_carteira_passivo'] =np.log(spread['Carteira_Credito'])   /np.log(spread['Passivo_Circulante']) 

In [25]:
%%time

spread.fillna(0,inplace=True)

Wall time: 42 ms


<a name="descricao"></a>

## 1.3) Descrição dos dados
- ir para [Menu Principal](#navegacao)
- ir para [Random Search Tunning](#randomsearch)
- ir para [Train/Test partition](#partition)

In [26]:
%%time

spread.filter(regex='firma', axis=1).describe().transpose()

Wall time: 140 ms


,count,mean,std,min,25%,50%,75%,max
firma_enc,59097.0,33605.668257,17273.775945,1.000000e+00,6909.000000,43792.000000,43792.000000,43792.000000
firma_potot,59097.0,3.373903,1.827505,4.987542e-03,1.951040,3.063391,4.617672,9.556813
firma_idade,59097.0,2.703713,0.840707,0.000000e+00,2.149434,2.761907,3.311273,4.390367
firma_massa,59097.0,10.781991,2.256507,0.000000e+00,9.172427,10.538891,12.404739,18.422188
firma_po3gr,59097.0,1.426054,1.609216,0.000000e+00,0.000000,0.747635,2.475277,9.104200
firma_rotat,59097.0,0.218042,0.103674,0.000000e+00,0.154151,0.223144,0.287682,0.405465
firma_engen,59097.0,0.240711,0.632251,0.000000e+00,0.000000,0.000000,0.000000,5.568345
firma_share,59097.0,0.001115,0.004967,5.847944e-09,0.000011,0.000062,0.000378,0.252814
firma_potot_idade,59097.0,1.203585,0.589484,1.954530e-03,0.783232,1.156933,1.551576,7.214130
firma_potot_massa,59097.0,0.296870,0.187812,2.096026e-03,0.212821,0.291238,0.376063,6.600389


In [27]:
%%time
spread.filter(regex='banco', axis=1).corr()

Wall time: 70 ms


,banco_enc,banco_b1,banco_b2,banco_b3,banco_ativo,banco_carteira,banco_passivo,banco_captacoes,banco_patrimonio,banco_lucroliquido,banco_numagencias,banco_numeropostos,banco_ativo_carteira,banco_ativo_passivo,banco_ativo_captacoes,banco_ativo_patrimonio,banco_carteira_passivo
banco_enc,1.000000,-0.532766,0.447746,0.075236,-0.516485,-0.444404,-0.516057,-0.515313,-0.550736,-0.154619,-0.527102,-0.515653,-0.395987,0.216522,0.173180,0.085373,0.490176
banco_b1,-0.532766,1.000000,-0.923967,-0.074246,0.858257,0.788242,0.854955,0.854562,0.830789,0.733292,0.909108,0.878072,0.537921,-0.294367,-0.226798,0.230219,-0.669508
banco_b2,0.447746,-0.923967,1.000000,-0.048411,-0.768582,-0.679683,-0.754886,-0.752924,-0.762391,-0.718387,-0.840101,-0.811309,-0.594165,0.096456,0.038595,-0.113488,0.638172
banco_b3,0.075236,-0.074246,-0.048411,1.000000,-0.115770,-0.251188,-0.110988,-0.119278,-0.159035,-0.067032,-0.079617,-0.065193,0.651952,-0.018025,0.089391,0.284931,-0.475047
banco_ativo,-0.516485,0.858257,-0.768582,-0.115770,1.000000,0.973998,0.998360,0.997361,0.981031,0.784405,0.959734,0.953875,0.453633,-0.385039,-0.296069,0.193882,-0.621366
banco_carteira,-0.444404,0.788242,-0.679683,-0.251188,0.973998,1.000000,0.973470,0.973936,0.952833,0.740434,0.912584,0.912680,0.249274,-0.392107,-0.320977,0.183471,-0.444345
banco_passivo,-0.516057,0.854955,-0.754886,-0.110988,0.998360,0.973470,1.000000,0.999410,0.973585,0.775316,0.954392,0.947651,0.451194,-0.437083,-0.346099,0.224979,-0.634789
banco_captacoes,-0.515313,0.854562,-0.752924,-0.119278,0.997361,0.973936,0.999410,1.000000,0.971967,0.769555,0.955277,0.948278,0.443518,-0.443408,-0.363655,0.227505,-0.630422
banco_patrimonio,-0.550736,0.830789,-0.762391,-0.159035,0.981031,0.952833,0.973585,0.971967,1.000000,0.793420,0.950461,0.950292,0.435220,-0.294014,-0.213387,0.002162,-0.580847
banco_lucroliquido,-0.154619,0.733292,-0.718387,-0.067032,0.784405,0.740434,0.775316,0.769555,0.793420,1.000000,0.755654,0.739969,0.438623,-0.182488,-0.078307,0.025071,-0.529737


In [28]:
%%time
spread.filter(regex='oper', axis=1).corr()

Wall time: 58 ms


,oper_valor,oper_spreadbndes,oper_juros,oper_amortizacao,oper_carencia,oper_valor_spreadbndes,oper_valor_juros,oper_valor_amortizacao,oper_valor_carencia,oper_spreadbndes_juros,oper_spreadbndes_amortizacao,oper_spreadbndes_carencia,oper_juros_amortizacao,oper_juros_carencia,oper_amortizacao_carencia
oper_valor,1.000000,0.001894,-0.082998,-0.017999,0.119088,0.963103,0.442593,0.077965,0.171839,0.087102,-0.006925,-0.124685,-0.021884,-0.143618,-0.028519
oper_spreadbndes,0.001894,1.000000,0.439267,-0.280726,0.078473,-0.267308,-0.394568,0.287715,-0.045006,-0.340261,0.310018,0.037314,0.341931,0.212503,-0.299802
oper_juros,-0.082998,0.439267,1.000000,-0.468580,-0.037531,-0.198199,-0.930363,0.450964,0.066276,-0.994218,0.466535,0.128063,0.600958,0.635669,-0.430714
oper_amortizacao,-0.017999,-0.280726,-0.468580,1.000000,0.098544,0.058209,0.415019,-0.946958,-0.102671,0.456919,-0.948220,-0.121443,-0.941217,-0.340641,0.914893
oper_carencia,0.119088,0.078473,-0.037531,0.098544,1.000000,0.093635,0.077585,-0.098277,-0.879206,0.048665,-0.106100,-0.914776,-0.104875,-0.735148,-0.153872
oper_valor_spreadbndes,0.963103,-0.267308,-0.198199,0.058209,0.093635,1.000000,0.532680,-0.002305,0.177699,0.175508,-0.090110,-0.130190,-0.113113,-0.195584,0.053206
oper_valor_juros,0.442593,-0.394568,-0.930363,0.415019,0.077585,0.532680,1.000000,-0.377105,0.003584,0.926670,-0.422349,-0.161107,-0.548810,-0.624831,0.377076
oper_valor_amortizacao,0.077965,0.287715,0.450964,-0.946958,-0.098277,-0.002305,-0.377105,1.000000,0.133850,-0.437644,0.996137,0.135570,0.981189,0.341188,-0.958854
oper_valor_carencia,0.171839,-0.045006,0.066276,-0.102671,-0.879206,0.177699,0.003584,0.133850,1.000000,-0.074752,0.117701,0.952575,0.120064,0.778515,0.153003
oper_spreadbndes_juros,0.087102,-0.340261,-0.994218,0.456919,0.048665,0.175508,0.926670,-0.437644,-0.074752,1.000000,-0.451277,-0.129586,-0.588166,-0.639969,0.415003


<a name="partition"></a>
    
## 1.4) Train/Test partition 

- ir para [Menu Principal](#navegacao)

In [29]:
%%time
files = spread.columns
selected_files = files.str.contains('firma|oper|banco')
features = files[selected_files]
lista = list(features)
#lista.remove('prazo_carencia')
lista.sort()

Wall time: 999 µs


In [30]:
lista

['banco_ativo',
 'banco_ativo_captacoes',
 'banco_ativo_carteira',
 'banco_ativo_passivo',
 'banco_ativo_patrimonio',
 'banco_b1',
 'banco_b2',
 'banco_b3',
 'banco_captacoes',
 'banco_carteira',
 'banco_carteira_passivo',
 'banco_enc',
 'banco_lucroliquido',
 'banco_numagencias',
 'banco_numeropostos',
 'banco_passivo',
 'banco_patrimonio',
 'firma_enc',
 'firma_engen',
 'firma_idade',
 'firma_idade_engen',
 'firma_idade_massa',
 'firma_idade_po3gr',
 'firma_massa',
 'firma_massa_po3gr',
 'firma_po3gr',
 'firma_potot',
 'firma_potot_idade',
 'firma_potot_massa',
 'firma_potot_po3gr',
 'firma_rotat',
 'firma_share',
 'oper_amortizacao',
 'oper_amortizacao_carencia',
 'oper_carencia',
 'oper_juros',
 'oper_juros_amortizacao',
 'oper_juros_carencia',
 'oper_spreadbndes',
 'oper_spreadbndes_amortizacao',
 'oper_spreadbndes_carencia',
 'oper_spreadbndes_juros',
 'oper_valor',
 'oper_valor_amortizacao',
 'oper_valor_carencia',
 'oper_valor_juros',
 'oper_valor_spreadbndes']

In [31]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
#from sklearn.experimental import enable_halving_search_cv 
#from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV

X, y = spread[lista] , spread[['target']]

Xmin_max = MinMaxScaler().fit_transform(X)
ymin_max = MinMaxScaler().fit_transform(y)

In [32]:
import sklearn
print(sklearn.__version__)

0.24.2


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X , y , test_size=0.30)

X_train0, X_test0, y_train0, y_test0 = train_test_split(Xmin_max , ymin_max , test_size=0.30)

<a name="bayesian"></a>

# 2) Bayesian Optimization for Hyperparameters Tunning

- 2.1 [Suport Vector Machine](#svm0)
- 2.2 [Stochastic Gradient Descent](#sgd0)
- 2.3 [Multi-Task Elastic-Net](#elastic0)
- 2.4 [Least Angle Regression Model (LAR)](#LAR0)
- 2.5 [Ridge Regression](#ridge0)
- 2.6 [ElasticNet Regression](#ElasNet0)
- 2.7 [Bayesian Ridge](#bayesian0)
- 2.8 [Lasso Regression](#lasso0)
- 2.9 [Multilayer Perceptron Regression](#mlp0)
- 2.10 [Nearest Neighbors Regression](#nnr0)
- 2.11 [Resumo dos resultados](#resumo_rs0)


<a name="svm0"></a>

## 2.1) Suport Vector Machine

- ir para [Menu Principal](#navegacao)

In [ ]:
%%time
from bayes_opt import BayesianOptimization
from sklearn.svm import SVR
from sklearn.svm import SVC

def gbm_svm(coef0,C,epsilon,degree,tol,shrinking):
    params_svm = {}
    params_svm['coef0']     = coef0
    params_svm['C']         = C
    params_svm['epsilon']   = epsilon 
    params_svm['degree']    = round(degree)
    params_svm['tol']       = tol
    params_svm['shrinking'] = round(shrinking)
    scores = cross_val_score( SVR(**params_svm), X_train, y_train, cv=3).mean()
    score = scores.mean()
    return score
    
# Run Bayesian Optimization
params_svm = {
  'epsilon'   : (0.01 ,3.5 ),
  'coef0'     : (0.01 ,3.5 ),
  'degree'    : (1    ,7   ),
  'C'         : (0.01 ,3.5 ),
  'tol'       : (0.001,0.2 ),
  'shrinking' : (False,True) 
}


svm0 = BayesianOptimization(gbm_svm, params_svm)

svm0.maximize(init_points=10, n_iter=50, acq='ucb')

|   iter    |  target   |     C     |   coef0   |  degree   |  epsilon  | shrinking |    tol    |
-------------------------------------------------------------------------------------------------
|  1        |  0.03673  |  0.5531   |  3.212    |  1.116    |  1.689    |  0.4171   |  0.06452  |
|  2        |  0.03141  |  3.377    |  0.2966   |  1.279    |  0.9797   |  0.1216   |  0.01626  |
|  3        | -0.01109  |  0.1497   |  1.724    |  5.142    |  2.923    |  0.5044   |  0.04818  |
|  4        | -0.02069  |  0.0222   |  0.9897   |  5.417    |  1.437    |  0.9876   |  0.1736   |
|  5        | -0.000616 |  1.881    |  2.203    |  2.33     |  0.6417   |  0.07226  |  0.06875  |
|  6        | -0.05485  |  0.01     |  3.5      |  1.0      |  2.959    |  0.7427   |  0.01962  |
|  7        |  0.02446  |  1.712    |  0.9828   |  5.586    |  1.286    |  0.7346   |  0.008679 |
|  8        |  0.03225  |  3.414    |  0.1098   |  1.246    |  1.1      |  0.2306   |  0.1645   |
|  9        |  0.016

In [28]:
params_svm0 = svm0.max['params']
params_svm0

{'C': 0.8172990905637877,
 'coef0': 0.5667243703919904,
 'degree': 4.375290208671179,
 'epsilon': 2.3717326421633635,
 'shrinking': 0.2852664272834471,
 'tol': 0.1461376611633345}

In [25]:
%%time
from bayes_opt import BayesianOptimization
from sklearn.svm import SVR
from sklearn.svm import SVC
svm00= SVR(C=0.8173, coef0=0.5667, degree=4.3753, epsilon=2.37173, shrinking=0.2852664, tol= 0.14614) 
svm00.fit(X_train, y_train)

CPU times: user 3min 16s, sys: 530 ms, total: 3min 16s
Wall time: 3min 17s


In [27]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test ,svm00.predict(X_test)))
print("EVS", explained_variance_score( y_test ,svm00.predict(X_test)))
print("MAE", mean_absolute_error(      y_test ,svm00.predict(X_test)))

R2: 0.05688974211166664
EVS 0.05868546521114004
MAE 2.2241919204935745


In [ ]:
import pickle
# Save to file in the current working directory
with open("pickle/svm00.pkl", 'wb') as file:
    pickle.dump(svm00, file)

<a name="sgd0"></a>

## 2.2) Stochastic Gradient Descent 

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [29]:
%%time
from bayes_opt import BayesianOptimization
from sklearn.linear_model import SGDRegressor

def gbm_sgd(alpha, l1_ratio, epsilon, eta0, tol, power_t, early_stopping):
    params_sgd = {}
    params_sgd['alpha']          = alpha.mean()
    params_sgd['l1_ratio']       = l1_ratio.mean()
    params_sgd['epsilon']        = epsilon .mean()
    params_sgd['eta0']           = eta0.mean()
    params_sgd['tol']            = tol.mean()
    params_sgd['power_t']        = power_t.mean()
    params_sgd['early_stopping'] = bool(early_stopping.mean())
    
    scores = cross_val_score(SGDRegressor(**params_sgd,max_iter=1000), X_train0, y_train0.ravel(), cv=3).mean()
    score = scores.mean()
    return score
    
# Run Bayesian Optimization
params_sgd = {
    'alpha'          : (0.050 ,10.0 ),
    'l1_ratio'       : (0.010 ,0.99 ),
    'epsilon'        : (0.010 ,0.99 ),
    'eta0'           : (0.001 ,0.99 ),
    'power_t'        : (0.010 ,0.99 ),
    'early_stopping' : (True  ,False),
    'tol'            : (0.001 ,0.99 )
}
#Nova senha RH Marilia1!2@3#

sgd0 = BayesianOptimization(gbm_sgd, params_sgd)

sgd0.maximize(init_points=10, n_iter=30, acq='ucb')

|   iter    |  target   |   alpha   | early_... |  epsilon  |   eta0    | l1_ratio  |  power_t  |    tol    |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.0741   |  5.479    |  0.6701   |  0.5473   |  0.1295   |  0.1251   |  0.2132   |  0.6123   |
|  2        | -141.3    |  8.575    |  0.1026   |  0.04378  |  0.9752   |  0.5991   |  0.9867   |  0.5813   |
|  3        |  0.08372  |  4.126    |  0.5894   |  0.05489  |  0.3954   |  0.6866   |  0.3359   |  0.1702   |
|  4        |  0.05401  |  6.764    |  0.6511   |  0.03897  |  0.5642   |  0.08227  |  0.4226   |  0.01225  |
|  5        |  0.06959  |  8.313    |  0.951    |  0.8103   |  0.05788  |  0.181    |  0.6875   |  0.03982  |
|  6        |  0.07028  |  8.411    |  0.01519  |  0.2135   |  0.13     |  0.2238   |  0.8146   |  0.5248   |
|  7        |  0.05793  |  9.712    |  0.5531   |  0.05063  |  0.3979   |  0.6291   |  0.5009   |  0.8994   |
|  8      

In [30]:
params_sgd0 = sgd0.max['params']
params_sgd0

{'alpha': 3.156269755184773,
 'early_stopping': 0.0,
 'epsilon': 0.15560487242834,
 'eta0': 0.06001768654406315,
 'l1_ratio': 0.872848897633387,
 'power_t': 0.4595269389751513,
 'tol': 0.3441389932404222}

In [68]:
%%time
from sklearn.linear_model import SGDRegressor
sgd00= SGDRegressor(random_state=1, max_iter=1000, alpha=3.156269755184773, l1_ratio=0.872848897633387,
                    epsilon=0.15560487242834, eta0=0.06001768654406315, power_t=0.4595269389751513, 
                    early_stopping=False, tol=0.3441389932404222)

sgd00.fit(X_train0, y_train0)

CPU times: user 103 ms, sys: 0 ns, total: 103 ms
Wall time: 100 ms


In [69]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0 ,sgd00.predict(X_test0)))
print("EVS", explained_variance_score( y_test0 ,sgd00.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0 ,sgd00.predict(X_test0)))

R2: 0.1465717173919352
EVS 0.14663557833907825
MAE 0.11311702316890586


In [72]:
importance = sgd00.coef_

In [79]:
#sgd00.get_params()
#sgd00.average_intercept_

In [71]:
dataframe=pd.DataFrame(sgd00.feature_importances_, columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

AttributeError: 'SGDRegressor' object has no attribute 'feature_importances_'

In [ ]:
SGDRegressor.get

<a name="elastic0"></a>

## 2.3) Multi-Task Elastic-Net

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from bayes_opt import BayesianOptimization
from sklearn import linear_model

#fit_intercept,normalize,warm_start  
#params_elastic['fit_intercept']  = True
#params_elastic['normalize']      = True
#params_elastic['warm_start']     = True

def gbm_elastic(alpha,l1_ratio):
    params_elastic = {}
    params_elastic['alpha']          = alpha
    params_elastic['l1_ratio']       = l1_ratio 
    scores = cross_val_score( linear_model.MultiTaskElasticNet(**params_elastic,max_iter=900), 
                             X_train0, y_train0.ravel(), cv=3).mean()
    score = scores.mean()
    return score
    
# Run Bayesian Optimization
params_elastic = {
    'alpha'         : (10.00 ,0.001),
    'l1_ratio'      : (0.999 ,0.001)
}

#'fit_intercept' : (0.010 ,0.99),
#'normalize'     : (0.010 ,0.99),
#'warm_start'    : (0.010 ,0.99),
    
mtask0 = BayesianOptimization(gbm_elastic, params_elastic)

mtask0.maximize(init_points=10, n_iter=30, acq='ucb')

In [ ]:
params_mtask0 = mtask0.max['params']
params_mtask0

In [ ]:
%%time
mtask00 = linear_model.MultiTaskElasticNet(max_iter=800,random_state=1, n_jobs=20)
mtask00.fit(X_train0, y_train0.ravel())

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test ,svm00.predict(X_test)))
print("EVS", explained_variance_score( y_test ,svm00.predict(X_test)))
print("MAE", mean_absolute_error(      y_test ,svm00.predict(X_test)))

In [34]:

# check xgboost version
import xgboost
print(xgboost.__version__)

1.3.3


<a name="LAR0"></a>

## 2.4) Least Angle Regression Model (LAR)

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [37]:
%%time
from sklearn import linear_model


def gbm_lars(alpha,fit_intercept,normalize,fit_path,eps,copy_X, positive):
    params_lars = {}
    params_lars['alpha']          = alpha
    params_lars['fit_intercept']  = fit_intercept
    params_lars['normalize']      = normalize 
    params_lars['fit_path']       = fit_path
    params_lars['eps']            = eps 
    params_lars['copy_X']         = copy_X 
    params_lars['positive']       = positive 
    
    scores = cross_val_score(linear_model.LassoLars(**params_lars,max_iter=800), 
                             X_train0, y_train0.ravel(), cv=3).mean()
    score = scores.mean()
    return score
    
# Run Bayesian Optimization
params_lars = {
    'alpha'         : (0.010 ,10.0),
    'fit_intercept' : (True,False) ,
    'normalize'     : (True,False) ,
    'fit_path'      : (True,False) ,
    'eps'           : (0.100 ,99.0),
    'copy_X'        : (True,False) ,
    'positive'      : (True,False) 
}


lars0 = BayesianOptimization(gbm_lars, params_lars)

lars0.maximize(init_points=10, n_iter=30, acq='ucb')

|   iter    |  target   |   alpha   |  copy_X   |    eps    | fit_in... | fit_path  | normalize | positive  |
-------------------------------------------------------------------------------------------------------------
|  1        | -6.297e-0 |  9.958    |  0.9492   |  6.053    |  0.4397   |  0.7421   |  0.2016   |  0.2105   |
|  2        | -6.297e-0 |  6.75     |  0.414    |  16.05    |  0.5728   |  0.3626   |  0.1632   |  0.05534  |
|  3        | -6.297e-0 |  9.367    |  0.1659   |  59.56    |  0.6793   |  0.7293   |  0.9501   |  0.7056   |
|  4        | -6.297e-0 |  1.757    |  0.5311   |  68.23    |  0.1085   |  0.4901   |  0.6827   |  0.79     |
|  5        | -6.297e-0 |  5.377    |  0.2627   |  77.41    |  0.9312   |  0.9715   |  0.2101   |  0.7024   |
|  6        | -6.297e-0 |  2.46     |  0.7848   |  90.99    |  0.01882  |  0.9563   |  0.04494  |  0.6115   |
|  7        | -6.297e-0 |  9.767    |  0.1519   |  1.459    |  0.7934   |  0.2845   |  0.683    |  0.6532   |
|  8      

In [38]:
params_lars0 = lars0.max['params']
params_lars0

{'alpha': 9.957909242789702,
 'copy_X': 0.9492126138746485,
 'eps': 6.053286348208411,
 'fit_intercept': 0.43967516680987495,
 'fit_path': 0.7420691416692062,
 'normalize': 0.20155510634876805,
 'positive': 0.210502974508331}

In [80]:
%%time
from sklearn import linear_model
lars00 = linear_model.LassoLars(max_iter=800, random_state=1, alpha= 0.014875, copy_X=False, eps=73.28342263,
                                fit_intercept= False, fit_path=False, normalize=False, positive=False)
lars00.fit(X_train0, y_train0.ravel())

CPU times: user 95.8 ms, sys: 55.8 ms, total: 152 ms
Wall time: 33.4 ms


In [81]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0 ,lars00.predict(X_test0)))
print("EVS", explained_variance_score( y_test0 ,lars00.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0 ,lars00.predict(X_test0)))

R2: 0.36947836589703864
EVS 0.37686817702026687
MAE 0.0942629446489631


In [90]:
#len(lars00.coef_[0])
len(lista)

47

In [93]:
dataframe=pd.DataFrame(lars00.coef_[0], columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

,feature_importances,features
0,0.000000,banco_ativo
24,0.000000,firma_massa_po3gr
25,0.000000,firma_po3gr
26,0.000000,firma_potot
27,0.000000,firma_potot_idade
28,0.000000,firma_potot_massa
29,0.000000,firma_potot_po3gr
30,0.000000,firma_rotat
31,0.000000,firma_share
32,0.000000,oper_amortizacao


In [ ]:
space = dict()
space['hidden_layer_sizes']  = [(100,50), (200,25), (25,100,25), (50,50,50), (75,50,75), (80,60,80)]
space['activation']          = ['identity','logistic','tanh','relu']
space['solver']              = ['lbfgs', 'sgd', 'adam']

space['learning_rate']       = ['constant','invscaling','adaptive']
space['learning_rate_init']  = [row/1000 for row in range(1,20,8)]

space['tol']                 = [row/50000000 for row in range(1,1000,200)]

space['nesterovs_momentum']  = [False,True]
space['early_stopping']      = [False,True]
space['warm_start']          = [False,True]

<a name="mlp0"></a>

## 2.9) Multilayer Perceptron

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Halving Random Serach](#halving)

In [89]:
%%time
from sklearn.neural_network import MLPRegressor

def gbm_mlp(alpha,power_t,momentum,beta_1,beta_2,epsilon,tol):
    params_lars = {}
    params_lars['alpha']             = alpha
    params_lars['power_t']           = power_t 
    params_lars['momentum']          = momentum 
    params_lars['beta_1']            = beta_1 
    params_lars['beta_2']            = beta_2 
    params_lars['epsilon']           = epsilon 
    params_lars['tol']               = tol 
    
    scores = cross_val_score(MLPRegressor(**params_lars,max_iter=800), X_train0, y_train0.ravel(), cv=3).mean()
    score = scores.mean()
    return score

# Run Bayesian Optimization
params_mlp = {
    'tol'                 : (2e-08,2e-05),
    'power_t'             : (9.999,0.0001),
    'momentum'            : (0.999,0.0001),
    'alpha'               : (0.999,0.0001),
    'beta_1'              : (0.999,0.0001),
    'beta_2'              : (0.999,0.0001),
    'epsilon'             : (0.999,0.0001)
}


mlp0 = BayesianOptimization(gbm_mlp, params_mlp)

mlp0.maximize(init_points=10, n_iter=30, acq='ucb')

|   iter    |  target   |   alpha   |  beta_1   |  beta_2   |  epsilon  | momentum  |  power_t  |    tol    |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.9747   |  0.7339   |  0.6946   |  0.5838   |  0.7037   |  0.9614   |  6.844    |  2.903e-0 |
|  2        |  0.8965   |  0.09487  |  0.3719   |  0.9375   |  0.6686   |  0.8558   |  5.055    |  1.976e-0 |
|  3        |  0.9701   |  0.8036   |  0.971    |  0.05772  |  0.3774   |  0.5397   |  8.967    |  1.533e-0 |
|  4        |  0.9815   |  0.09038  |  0.9287   |  0.8397   |  0.1864   |  0.9622   |  2.024    |  9.153e-0 |
|  5        |  0.9729   |  0.6376   |  0.7507   |  0.4202   |  0.9246   |  0.5264   |  1.473    |  1.114e-0 |
|  6        |  0.9863   |  0.1605   |  0.2694   |  0.5937   |  0.3598   |  0.387    |  3.843    |  7.571e-0 |
|  7        |  0.98     |  0.6001   |  0.871    |  0.906    |  0.478    |  0.2696   |  6.017    |  1.287e-0 |
|  8      

In [ ]:
params_mlp0 = mlp0.max['params']
params_mlp0

In [ ]:
%%time
lars00 = MLPRegressor(**params_lars,max_iter=800)
(max_iter=800, random_state=1, alpha= 0.014875, copy_X=False, eps=73.28342263,
                                fit_intercept= False, fit_path=False, normalize=False, positive=False)
lars00.fit(X_train0, y_train0.ravel())

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0 ,mlp0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0 ,mlp0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0 ,mlp0.predict(X_test0)))

<a name="feature0"></a>

## 2.10) Feature Importances

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Halving Random Serach](#halving)

In [ ]:
dataframe=pd.DataFrame(model.feature_importances_, columns=['feature_importances']) 

In [ ]:
dataframe=pd.DataFrame(model.feature_importances_, columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

<a name="randomsearch"></a>

# 3) Random Search for Hyperparameters

- 3.1 [Extreme Learning Machine](#elm1)
- 3.2 [Suport Vector Machine](#svm1)
- 3.3 [Stochastic Gradient Descent](#sgd1)
- 3.4 [Multi-Task Elastic-Net](#elastic1)
- 3.5 [Least Angle Regression Model (LAR)](#LAR1)
- 3.6 [Ridge Regression](#ridge1)
- 3.7 [ElasticNet Regression](#ElasNet1)
- 3.8 [Bayesian Ridge](#bayesian1)
- 3.9 [Lasso Regression](#lasso1)
- 3.10 [Multilayer Perceptron Regression](#mlp1)
- 3.11 [Nearest Neighbors Regression](#nnr1)
- 3.12 [Resumo dos resultados](#resumo_rs1)

<a name="svm1"></a>

## 3.1) Suport Vector Machine

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)


In [42]:
%%time
from sklearn.svm import SVR
from sklearn.svm import SVC

# define search space
space = dict()
space['kernel']    = ['linear','poly','rbf','sigmoid']
space['gamma']     = ['scale','auto']
space['coef0']     = [row/50 for row in range(1,100,4)]
space['C']         = [row/50 for row in range(1,100,4)]
space['epsilon']   = [row/25 for row in range(5,25)]
space['degree']    = [row/500 for row in range(1,250,5)]
space['tol']       = [row/2000 for row in range(1,200,10)]
space['shrinking'] = [True, False] 

svm = SVR()

svm1= RandomizedSearchCV(svm, space ,  cv=3, random_state=1, n_iter=300, n_jobs=20)

svm1.fit(X_train, y_train)

KeyboardInterrupt: 

In [43]:
# summarize result
print('Best Score: %s' % svm1.best_score_) #Best Score: 0.9352753839517703
print('Best Hyperparameters: %s' % svm1.best_params_)

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_score_'

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0 ,svm1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0 ,svm1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0 ,svm1.predict(X_test0)))

In [ ]:
import pickle
# Save to file in the current working directory
with open("pickle/svm0.pkl", 'wb') as file:
    pickle.dump(svm1, file)

In [ ]:
dataframe=pd.DataFrame(model.feature_importances_, columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

<a name="sgd1"></a>

## 3.2) Stochastic Gradient Descent 

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.linear_model import SGDRegressor

# define search space
space = dict()
space['alpha']          = [row/50 for row in range(40,500,25)]
space['l1_ratio']       = [row/400 for row in range(20,100,4)] 
space['loss']           = ['squared_loss','huber','epsilon_insensitive','squared_epsilon_insensitive']
space['penalty']        = ['l1','l2','elasticnet']
space['learning_rate']  = ['invscaling','constant','optimal','adaptive']
space['epsilon']        = [row/700 for row in range(10,200,20)]
space['eta0']           = [row/1200 for row in range(1,200,10)]
space['power_t']        = [row/600 for row in range(60,300,10)]
space['early_stopping'] = [False,True]
space['tol']            = [row/5000 for row in range(1,60,5)]

sgd = SGDRegressor(max_iter=1000)

sgd0 = RandomizedSearchCV(sgd, space, cv=3, random_state=666, n_iter=300, n_jobs=20)

sgd0.fit(X_train0, y_train0.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % sgd0.best_score_) 
print('Best Hyperparameters: %s' % sgd0.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0.ravel(),sgd0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0.ravel(),sgd0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0.ravel(),sgd0.predict(X_test0)))

In [ ]:
import pickle

# Save to file in the current working directory
with open("pickle/sgd0.pkl", 'wb') as file:
    pickle.dump(sgd0, file)

<a name="elastic1"></a>

## 3.3) Multi-Task Elastic-Net

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [28]:
%%time
from sklearn import linear_model

space = dict()
space['alpha']         = [row/200 for row in range(1,1200,60)]
space['l1_ratio']      = [row/30 for row in range(1,300,10)]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['warm_start']    = [False,True]
space['copy_X']        = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/5000 for row in range(1,20,2)]

mtask = linear_model.MultiTaskElasticNet(max_iter=800)

mtask0 = RandomizedSearchCV(mtask, space, cv=3,  n_iter=300, random_state=1, n_jobs=20)

mtask0.fit(X_train, y_train)

CPU times: user 56min 36s, sys: 1h 35min 21s, total: 2h 31min 58s
Wall time: 17min 7s


In [29]:
# summarize result
print('Best Score: %s' % mtask0.best_score_)
print('Best Hyperparameters: %s' % mtask0.best_params_)

Best Score: 0.9909882517195915
Best Hyperparameters: {'warm_start': True, 'tol': 0.0006, 'selection': 'cyclic', 'normalize': False, 'l1_ratio': 1.0333333333333334, 'fit_intercept': False, 'copy_X': False, 'alpha': 0.005}


In [30]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,mtask0.predict(X_test)))
print("EVS", explained_variance_score( y_test,mtask0.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,mtask0.predict(X_test)))

R2: 0.9908510042915026
EVS 0.9908510161021391
MAE 0.2233163826291033


In [31]:
mtask00 = linear_model.MultiTaskElasticNet(max_iter=800, warm_start=True, tol=0.0006, selection='cyclic',normalize=False,
                                           l1_ratio=1.0333333333333334, fit_intercept=False, copy_X=False, alpha=0.005)


mtask00.fit(X_train, y_train)

MultiTaskElasticNet(alpha=0.005, copy_X=False, fit_intercept=False,
                    l1_ratio=1.0333333333333334, max_iter=800, tol=0.0006,
                    warm_start=True)

In [32]:
import pickle
# Save to file in the current working directory
with open("pickle/mtask00.pkl", 'wb') as file:
    pickle.dump(mtask00, file)

In [35]:
dataframe=pd.DataFrame(mtask00.feature_importances_, columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

AttributeError: 'MultiTaskElasticNet' object has no attribute 'feature_importances_'

<a name="LAR1"></a>

## 3.4) Least Angle Regression Model (LAR)

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [36]:
%%time
from sklearn import linear_model

space = dict()
space['alpha']          = [row/300 for row in range(1,900,10)]
space['fit_intercept']  = [False,True]
space['fit_path']       = [False,True]
space['normalize']      = [False,True]
space['copy_X']         = [False,True]
space['positive']       = [False,True]
space['eps']            = [row/100 for row in range(1000,10000,100)]

lars = linear_model.LassoLars()

lars0 = RandomizedSearchCV(lars, space, cv=3,  n_iter=300, random_state=666, n_jobs=20)

lars0.fit(X_train, y_train)

CPU times: user 17.8 s, sys: 399 ms, total: 18.2 s
Wall time: 41.6 s


In [37]:
# summarize result
print('Best Score: %s' % lars0.best_score_) #Best Score: 0.9471256547797681
print('Best Hyperparameters: %s' % lars0.best_params_)

Best Score: 0.9909092927498436
Best Hyperparameters: {'positive': False, 'normalize': True, 'fit_path': False, 'fit_intercept': False, 'eps': 22.0, 'copy_X': False, 'alpha': 0.03666666666666667}


In [38]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test,lars0.predict(X_test)))
print("EVS", explained_variance_score( y_test,lars0.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,lars0.predict(X_test)))

R2: 0.990748285663413
EVS 0.9907485154304116
MAE 0.2277757851876471


In [39]:
lars00 = linear_model.LassoLars(max_iter=800, positive=False, normalize=True, fit_path=False,
                                fit_intercept= False, eps= 22.0, copy_X= False, alpha= 0.0366667)
    
lars00.fit(X_train, y_train)
import pickle
# Save to file in the current working directory
with open("pickle/lars00.pkl", 'wb') as file:
    pickle.dump(lars00, file)

In [41]:
dataframe=pd.DataFrame(lars00.feature_importances_, columns=['feature_importances']) 
dataframe['features'] = lista
dataframe.sort_values('feature_importances')

AttributeError: 'LassoLars' object has no attribute 'feature_importances_'

In [ ]:
lars00.n_features_in_

<a name="ridge1"></a>

## 3.5) Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.linear_model import Ridge
  
space = dict()
space['alpha']         = [row/500 for row in range(8,100,5)]
space['copy_X']        = [False,True]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['solver']        = ['auto','svd','cholesky','lsqr','sparse_cg']
space['tol']           = [row/100000 for row in range(1,1000,10)]

ridge = Ridge()

ridge0 = RandomizedSearchCV(ridge, space, cv=3,  n_iter=300 , random_state=666, n_jobs=20)

ridge0.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % ridge0.best_score_) #Best Score: 0.9917155408859822
print('Best Hyperparameters: %s' % ridge0.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, ridge0.predict(X_test)))
print("EVS", explained_variance_score( y_test, ridge0.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, ridge0.predict(X_test)))

In [ ]:
import pickle

# Save to file in the current working directory
with open("pickle/ridge0.pkl", 'wb') as file:
    pickle.dump(ridge0, file)

<a name="ElasNet1"></a>

## 3.6) ElasticNet Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.linear_model import ElasticNet

space = dict()
space['alpha']         = [row/500 for row in range(1,500,5)]
space['l1_ratio']      = [row/250 for row in range(5,1000,10)]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['precompute']    = [False,True]
space['copy_X']        = [False,True]
space['warm_start']    = [False,True]
space['positive']      = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/10000 for row in range(10,10000,10)]

elastic = ElasticNet()

elastic0 = RandomizedSearchCV(elastic, space, cv=3,  n_iter=300, random_state=666, n_jobs=20)

elastic0.fit(X_train0, y_train0)

In [ ]:
# summarize result
print('Best Score: %s' % elastic0.best_score_) #Best Score: 0.6300884516001556
print('Best Hyperparameters: %s' % elastic0.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, elastic0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, elastic0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, elastic0.predict(X_test0)))

In [ ]:
import pickle

# Save to file in the current working directory
with open("pickle/elastic0.pkl", 'wb') as file:
    pickle.dump(elastic0, file)

<a name="bayesian1"></a>

## 3.7) Bayesian Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.linear_model import BayesianRidge

space = dict()
space['alpha_1']        = [row/40000000 for row in range(1,2000,50)]
space['alpha_2']        = [row/40000000 for row in range(1,2000,50)]
space['lambda_1']       = [row/40000000 for row in range(1,2000,50)]
space['lambda_2']       = [row/40000000 for row in range(1,2000,50)]
space['compute_score']  = [False,True]
space['copy_X']         = [False,True]
space['fit_intercept']  = [False,True]
space['normalize']      = [False,True]
space['tol']            = [row/5000 for row in range(1,100,5)]

bayes = BayesianRidge(n_iter=800)

bayes0 = RandomizedSearchCV(bayes, space, cv=3,  n_iter=300, random_state=666, n_jobs=20)

bayes0.fit(X_train0, y_train0.ravel())


In [ ]:
# summarize result
print('Best Score: %s' % bayes0.best_score_)
print('Best Hyperparameters: %s' % bayes0.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, bayes0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, bayes0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, bayes0.predict(X_test0)))

In [ ]:
import pickle

# Save to file in the current working directory
with open("pickle/bayes0.pkl", 'wb') as file:
    pickle.dump(bayes0, file)

<a name="lasso1"></a>

## 3.8) Lasso Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.linear_model import Lasso

space = dict()
space['alpha']         = [row/1000 for row in range(10,100,5)]
space['copy_X']        = [False,True]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['precompute']    = [False,True]
space['positive']      = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/1000000 for row in range(100,10000,10)]

lasso = Lasso()

lasso0 = RandomizedSearchCV(lasso, space, cv=3, n_iter=300, random_state=666, n_jobs=20)

lasso0.fit(X_train0, y_train0)

In [ ]:
# summarize result
print('Best Score: %s' % lasso0.best_score_) #Best Score: 0.8776039138486295
print('Best Hyperparameters: %s' % lasso0.best_params_) 

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, lasso0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, lasso0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, lasso0.predict(X_test0)))

In [ ]:
import pickle

# Save to file in the current working directory
with open("pickle/lasso0.pkl", 'wb') as file:
    pickle.dump(lasso0, file)

<a name="mlp1"></a>

## 3.9) Multilayer Perceptron

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.neural_network import MLPRegressor

space = dict()
space['hidden_layer_sizes']  = [(100,50), (200,25), (25,100,25), (50,50,50), (75,50,75), (80,60,80)]
space['activation']          = ['identity','logistic','tanh','relu']
space['solver']              = ['lbfgs', 'sgd', 'adam']
space['alpha']               = [row/200 for row in range(1,100,10)]
space['learning_rate']       = ['constant','invscaling','adaptive']
space['learning_rate_init']  = [row/1000 for row in range(1,20,8)]
space['warm_start']          = [False,True]
space['power_t']             = [row/50 for row in range(1,200,20)]
space['tol']                 = [row/50000000 for row in range(1,1000,200)]
space['momentum']            = [row/200 for row in range(1,100,10)]
space['nesterovs_momentum']  = [False,True]
space['early_stopping']      = [False,True]
space['beta_1']              = [row/200 for row in range(1,20,6)]
space['beta_2']              = [row/200 for row in range(1,20,6)]
space['epsilon']             = [row/200 for row in range(1,20,6)]
    
mpl = MLPRegressor(max_iter=1000)

mpl0 = RandomizedSearchCV(mpl, space, cv=3,  n_iter=300, random_state=666, n_jobs=20)

mpl0.fit(X_train0, y_train0.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % mpl0.best_score_) #Best Score: 0.9550165534562787
print('Best Hyperparameters: %s' % mpl0.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, mpl0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, mpl0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, mpl0.predict(X_test0)))

In [ ]:
import pickle

# Save to file in the current working directory
with open("pickle/mpl0.pkl", 'wb') as file:
    pickle.dump(mpl0, file)

<a name="nnr1"></a>

## 3.10) Nearest Neighbors Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.neighbors import KNeighborsRegressor

space = dict()
space['n_neighbors'] = [row for row in range(4,7)]
space['weights']     = ['uniform', 'distance']
space['algorithm']   = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size']   = [row for row in range(5,30)]
space['p']           = [row for row in range(2,7)]

knn = KNeighborsRegressor()

knn0 = RandomizedSearchCV(knn, space, cv=3, random_state=666, n_iter=300, n_jobs=20)

knn0.fit(X_train0, y_train0)

In [ ]:
# summarize result
print('Best Score: %s' % knn0.best_score_)
print('Best Hyperparameters: %s' % knn0.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, knn0.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, knn0.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, knn0.predict(X_test0)))

In [ ]:
import pickle

# Save to file in the current working directory
with open("pickle/knn0.pkl", 'wb') as file:
    pickle.dump(knn0, file)

<a name="resumo_rs1"></a>

## 3.11) Resumo dos resultados

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
R2_svm      = r2_score(                 y_test0 ,svm0.predict(X_test0))
EVS_svm     = explained_variance_score( y_test0 ,svm0.predict(X_test0))
MAE_svm     = mean_absolute_error(      y_test0 ,svm0.predict(X_test0))

R2_sgd      = r2_score(                 y_test0 ,sgd0.predict(X_test0))
EVS_sgd     = explained_variance_score( y_test0 ,sgd0.predict(X_test0))
MAE_sgd     = mean_absolute_error(      y_test0 ,sgd0.predict(X_test0))

R2_mtask    = r2_score(                 y_test  ,mtask0.predict(X_test))
EVS_mtask   = explained_variance_score( y_test  ,mtask0.predict(X_test))
MAE_mtask   = mean_absolute_error(      y_test  ,mtask0.predict(X_test))

R2_lars     = r2_score(                 y_test  ,lars0.predict(X_test))
EVS_lars    = explained_variance_score( y_test  ,lars0.predict(X_test))
MAE_lars    = mean_absolute_error(      y_test  ,lars0.predict(X_test))

R2_ridge    = r2_score(                 y_test  ,ridge0.predict(X_test))
EVS_ridge   = explained_variance_score( y_test  ,ridge0.predict(X_test))
MAE_ridge   = mean_absolute_error(      y_test  ,ridge0.predict(X_test))

R2_elastic  = r2_score(                 y_test0 ,elastic0.predict(X_test0))
EVS_elastic = explained_variance_score( y_test0 ,elastic0.predict(X_test0))
MAE_elastic = mean_absolute_error(      y_test0 ,elastic0.predict(X_test0))

R2_bayes    = r2_score(                 y_test0 ,bayes0.predict(X_test0))
EVS_bayes   = explained_variance_score( y_test0 ,bayes0.predict(X_test0))
MAE_bayes   = mean_absolute_error(      y_test0 ,bayes0.predict(X_test0))

R2_lasso    = r2_score(                 y_test0 ,lasso0.predict(X_test0))
EVS_lasso   = explained_variance_score( y_test0 ,lasso0.predict(X_test0))
MAE_lasso   = mean_absolute_error(      y_test0 ,lasso0.predict(X_test0))

R2_mpl      = r2_score(                 y_test0 ,mpl0.predict(X_test0))
EVS_mpl     = explained_variance_score( y_test0 ,mpl0.predict(X_test0))
MAE_mpl     = mean_absolute_error(      y_test0 ,mpl0.predict(X_test0))

R2_knn      = r2_score(                 y_test0 ,knn0.predict(X_test0))
EVS_knn     = explained_variance_score( y_test0 ,knn0.predict(X_test0))
MAE_knn     = mean_absolute_error(      y_test0 ,knn0.predict(X_test0))

In [ ]:
RSQ = [R2_svm , R2_sgd , R2_mtask , R2_lars , R2_lasso , R2_ridge , R2_elastic  , R2_bayes , R2_mpl , R2_knn]
EVS = [EVS_svm, EVS_sgd, EVS_mtask, EVS_lars, EVS_lasso, EVS_ridge, EVS_elastic , EVS_bayes, EVS_mpl, EVS_knn]
MAE = [MAE_svm, MAE_sgd, MAE_mtask, MAE_lars, MAE_lasso, MAE_ridge, MAE_elastic , MAE_bayes, MAE_mpl, MAE_knn]
MOD = ['SVM'  , 'SGD'  , 'MTASK'  , 'LARS'  , 'Lasso'  , 'Ridge'  ,'Elastic Net', 'Bayes'  , 'MLP'  , 'KNN']
BHY = [ svm0.best_params_ , sgd0.best_params_, mtask0.best_params_, lars0.best_params_, lasso0.best_params_,
        ridge0.best_params_, elastic0.best_params_, bayes0.best_params_, mpl0.best_params_, knn0.best_params_]

In [ ]:
resumo_rs = {'Mod' : MOD,
             'R2'  : RSQ,
             'EVS' : EVS,
             'MAE' : MAE,
             'BHY' : BHY }

In [ ]:
pd_resumo_rlv0 = pd.DataFrame(data=resumo_rs)
pd_resumo_rlv0

In [ ]:
pd_resumo_rlv0.to_csv('pd_resumo_rlv0.csv')

<a name="halving"></a>

# 4) Halving Random Serach Tunning

- 4.1 [Suport Vector Machine](#svm2)
- 4.2 [Stochastic Gradient Descent](#sgd2)
- 4.3 [Multi-Task Elastic-Net](#elastic2)
- 4.4 [Least Angle Regression Model (LAR)](#LAR2)
- 4.5 [Ridge Regression](#ridge2)
- 4.6 [ElasticNet Regression](#ElasNet2)
- 4.7 [Bayesian Ridge](#bayesian2)
- 4.8 [Lasso Regression](#lasso2)
- 4.9 [Multilayer Perceptron Regression](#mlp2)
- 4.10 [Nearest Neighbors Regression](#nnr2)
- 4.11 [Resumo dos resultados](#resumo_rs2)

<a name="svm2"></a>

## 4.1) Suport Vector Machine

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.svm import SVR

# define search space
space = dict()
space = dict()
space['kernel']    = ['linear','poly','rbf','sigmoid']
space['gamma']     = ['scale','auto']
space['coef0']     = [row/50 for row in range(1,100,4)]
space['C']         = [row/50 for row in range(1,100,4)]
space['epsilon']   = [row/25 for row in range(5,25)]
space['degree']    = [row/500 for row in range(1,250,5)]
space['tol']       = [row/2000 for row in range(1,200,10)]
space['shrinking'] = [True, False] 

svm  = SVR()

svm1 = HalvingRandomSearchCV(svm, space, n_candidates=300, random_state=666, cv=3, 
                             max_resources=600, factor=3, n_jobs=20, scoring='f1')

svm1.fit(X_train, y_train)
#4min 3s

In [ ]:
# summarize result
print('Best Score: %s' % svm1.best_score_)
print('Best Hyperparameters: %s' % svm1.best_params_)

In [ ]:
%%time
svm1= SVR(C= 3.2633, coef0 = 0.8874, degree= 1.6077, epsilon= 2.21599, shrinking=False, tol= 0.00825) 
svm1.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
print("R2:", r2_score(                 y_test0, svm1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, svm1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, svm1.predict(X_test0)))

In [ ]:
# Save to file in the current working directory
with open("pickle/svm1.pkl", 'wb') as file:
    pickle.dump(svm1, file)

<a name="sgd2"></a>

## 4.2) Stochastic Gradient Descent 

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.linear_model import SGDRegressor

# define search space
space = dict()
space['alpha']          = [row/50 for row in range(40,500,10)]
space['l1_ratio']       = [row/400 for row in range(20,100,8)] 
space['loss']           = ['squared_loss']
space['penalty']        = ['l2']
space['learning_rate']  = ['invscaling']
space['epsilon']        = [row/700 for row in range(10,200,10)]
space['eta0']           = [row/1200 for row in range(1,200,10)]
space['power_t']        = [row/600 for row in range(60,300,10)]
space['early_stopping'] = [False]
space['tol']            = [row/5000 for row in range(1,60,20)]

sgd = SGDRegressor(max_iter=1500)

sgd1 = HalvingRandomSearchCV(sgd, space, n_candidates=300, random_state=666, cv=3, max_resources=1000, 
                             factor=3, n_jobs=20, scoring='f1')


sgd1.fit(X_train0, y_train0.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % sgd1.best_score_)
print('Best Hyperparameters: %s' % sgd1.best_params_)

In [ ]:
%%time
svm1= SVR(C= 3.2633, coef0 = 0.8874, degree= 1.6077, epsilon= 2.21599, shrinking=False, tol= 0.00825) 
svm1.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, sgd1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, sgd1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, sgd1.predict(X_test0)))

In [ ]:
# Save to file in the current working directory
with open("pickle/sgd1.pkl", 'wb') as file:
    pickle.dump(sgd1, file)

<a name="elastic2"></a>

## 4.3) Multi-Task Elastic-Net

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn import linear_model

space = dict()
space['alpha']         = [row/200 for row in range(1,1200,60)]
space['l1_ratio']      = [row/30 for row in range(1,300,10)]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['warm_start']    = [False,True]
space['copy_X']        = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/5000 for row in range(1,20,2)]


mtask = linear_model.MultiTaskElasticNet(max_iter=800)

mtask1 = HalvingRandomSearchCV(mtask, space, n_candidates=300, random_state=666, cv=3, max_resources=1000, 
                             factor=3, n_jobs=20, scoring='f1')

mtask1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % mtask1.best_score_)
print('Best Hyperparameters: %s' % mtask1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,mtask1.predict(X_test)))
print("EVS", explained_variance_score( y_test,mtask1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,mtask1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/mtask1.pkl", 'wb') as file:
    pickle.dump(mtask1, file)

<a name="LAR2"></a>

## 4.4) Least Angle Regression Model (LAR)

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
linear_model.LassoLars().get_params().keys()

In [ ]:
%%time
from sklearn import linear_model

space = dict()
space['alpha']          = [row/300 for row in range(1,900,10)]
space['fit_intercept']  = [False,True]
space['fit_path']       = [False,True]
space['normalize']      = [False,True]
space['copy_X']         = [False,True]
space['positive']       = [False,True]
space['eps']            = [row/100 for row in range(1000,10000,100)]


lars = linear_model.LassoLars()

lars1 = HalvingRandomSearchCV(mtask, space, n_candidates=300, random_state=666, cv=3, max_resources=1000, 
                             factor=3, n_jobs=20, scoring='f1')

lars1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % lars1.best_score_)
print('Best Hyperparameters: %s' % lars1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test,lars1.predict(X_test)))
print("EVS", explained_variance_score( y_test,lars1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test,lars1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/lars1.pkl", 'wb') as file:
    pickle.dump(lars1, file)

<a name="ridge2"></a>

## 4.5) Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)

In [ ]:
%%time
from sklearn.linear_model import Ridge
space = dict()
space['alpha']         = [row/500 for row in range(8,100,5)]
space['copy_X']        = [False,True]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['solver']        = ['auto','svd','cholesky','lsqr','sparse_cg']
space['tol']           = [row/100000 for row in range(1,1000,10)]

ridge = Ridge()

ridge1 = HalvingRandomSearchCV(ridge, space, n_candidates=300, random_state=666, cv=3, factor=3, n_jobs=20, scoring='f1')

ridge1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % ridge1.best_score_)
print('Best Hyperparameters: %s' % ridge1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, ridge1.predict(X_test)))
print("EVS", explained_variance_score( y_test, ridge1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, ridge1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/ridge1.pkl", 'wb') as file:
    pickle.dump(ridge1, file)

<a name="ElasNet2"></a>

## 4.6) ElasticNet Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)



In [ ]:
%%time
from sklearn.linear_model import ElasticNet

space = dict()
space['alpha']         = [row/500 for row in range(1,500,5)]
space['l1_ratio']      = [row/250 for row in range(5,1000,10)]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['precompute']    = [False,True]
space['copy_X']        = [False,True]
space['warm_start']    = [False,True]
space['positive']      = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/10000 for row in range(10,10000,10)]

elastic = ElasticNet()

elastic1 = HalvingRandomSearchCV(elastic, space, n_candidates=300, random_state=666, cv=3, factor=3, n_jobs=20, scoring='f1')

elastic1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % elastic1.best_score_)
print('Best Hyperparameters: %s' % elastic1.best_params_)

In [ ]:
# summarize result
print('Best Score: %s' % elastic1.best_score_)
print('Best Hyperparameters: %s' % elastic1.best_params_)

In [ ]:
# Save to file in the current working directory
with open("pickle/elastic1.pkl", 'wb') as file:
    pickle.dump(elastic1, file)

<a name="bayesian2"></a>

## 4.7) Bayesian Ridge Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)


In [ ]:
%%time   
from sklearn.linear_model import BayesianRidge

space = dict()
space['alpha_1']        = [row/40000000 for row in range(1,2000,100)]
space['alpha_2']        = [row/40000000 for row in range(1,2000,100)]
space['lambda_1']       = [row/40000000 for row in range(1,2000,100)]
space['lambda_2']       = [row/40000000 for row in range(1,2000,100)]
space['compute_score']  = [False]
space['copy_X']         = [True]
space['fit_intercept']  = [True]
space['normalize']      = [False]
space['tol']            = [row/5000 for row in range(1,100,10)]

bayes = BayesianRidge()

bayes1 = HalvingRandomSearchCV(bayes, space, n_candidates = 300 , max_resources=50, 
                               random_state=666, cv=3, factor=3, n_jobs=20 , scoring='f1')
bayes1.fit(X_train, y_train.values.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % bayes1.best_score_)
print('Best Hyperparameters: %s' % bayes1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, bayes1.predict(X_test)))
print("EVS", explained_variance_score( y_test, bayes1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, bayes1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/bayes1.pkl", 'wb') as file:
    pickle.dump(bayes1, file)

<a name="lasso2"></a>

## 4.8) Lasso Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)



In [ ]:
%%time
from sklearn.linear_model import Lasso

space = dict()
space['alpha']         = [row/1000 for row in range(10,100,5)]
space['copy_X']        = [False,True]
space['fit_intercept'] = [False,True]
space['normalize']     = [False,True]
space['precompute']    = [False,True]
space['positive']      = [False,True]
space['selection']     = ['cyclic','random']
space['tol']           = [row/1000000 for row in range(100,10000,10)]


lasso = Lasso()

lasso1 = HalvingRandomSearchCV(lasso, space, n_candidates = 300 , max_resources=50, 
                               random_state=666, cv=3, factor=3, n_jobs=20 , scoring='f1')

lasso1.fit(X_train, y_train.values.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % lasso1.best_score_)
print('Best Hyperparameters: %s' % lasso1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, lasso1.predict(X_test)))
print("EVS", explained_variance_score( y_test, lasso1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, lasso1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/lasso1.pkl", 'wb') as file:
    pickle.dump(lasso1, file)

<a name="mlp2"></a>

## 4.9) Multilayer Perceptron

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)



In [ ]:
%%time
from sklearn.neural_network import MLPRegressor

space = dict()
space['hidden_layer_sizes']  = [(100,50), (25,100,25), (75,50,75)]
space['activation']          = ['identity','logistic','tanh','relu']
space['solver']              = ['lbfgs', 'sgd', 'adam']
space['alpha']               = [row/200 for row in range(1,100,20)]
space['learning_rate']       = ['constant','invscaling','adaptive']
space['learning_rate_init']  = [row/1000 for row in range(1,20,10)]
space['warm_start']          = [False]
space['power_t']             = [row/50 for row in range(1,200,40)]
space['tol']                 = [row/50000000 for row in range(1,1000,400)]
space['momentum']            = [row/200 for row in range(1,100,20)]
space['nesterovs_momentum']  = [True]
space['early_stopping']      = [False]
space['beta_1']              = [row/200 for row in range(1,20,12)]
space['beta_2']              = [row/200 for row in range(1,20,12)]
space['epsilon']             = [row/200 for row in range(1,20,12)]
    
mpl = MLPRegressor(max_iter=1200)

mpl1 = HalvingRandomSearchCV(mpl, space, n_candidates = 300 , max_resources=50, 
                               random_state=666, cv=3, factor=3, n_jobs=20 , scoring='f1')

mpl1.fit(X_train0, y_train0.ravel())

In [ ]:
# summarize result
print('Best Score: %s' % mpl1.best_score_)
print('Best Hyperparameters: %s' % mpl1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test0, mpl1.predict(X_test0)))
print("EVS", explained_variance_score( y_test0, mpl1.predict(X_test0)))
print("MAE", mean_absolute_error(      y_test0, mpl1.predict(X_test0)))

In [ ]:
# Save to file in the current working directory
with open("pickle/mpl1.pkl", 'wb') as file:
    pickle.dump(mpl1, file)

<a name="nnr2"></a>

## 4.10) Nearest Neighbors Regression

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)


In [ ]:
%%time
from sklearn.neighbors import KNeighborsRegressor

space = dict()
space['n_neighbors'] = [row for row in range(4,10,2)]
space['weights']     = ['uniform', 'distance']
space['algorithm']   = ['auto', 'ball_tree', 'kd_tree', 'brute']
space['leaf_size']   = [row for row in range(10,50,2)]
space['p']           = [row for row in range(2,8,2)]
    
knn = KNeighborsRegressor()

knn1 = HalvingRandomSearchCV(knn, space, n_candidates = 300 , max_resources=50, 
                               random_state=666, cv=3, factor=3, n_jobs=20 , scoring='f1')

knn1.fit(X_train, y_train)

In [ ]:
# summarize result
print('Best Score: %s' % knn1.best_score_)
print('Best Hyperparameters: %s' % knn1.best_params_)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error

print("R2:", r2_score(                 y_test, knn1.predict(X_test)))
print("EVS", explained_variance_score( y_test, knn1.predict(X_test)))
print("MAE", mean_absolute_error(      y_test, knn1.predict(X_test)))

In [ ]:
# Save to file in the current working directory
with open("pickle/knn1.pkl", 'wb') as file:
    pickle.dump(knn1, file)

<a name="resumo_rs2"></a>

## 4.11) Resumo dos resultados

- ir para [Menu Principal](#navegacao)
- ir para [Train/Test partition](#partition)
- ir para [Baysian Optimization](#bayesian)
- ir para [Random Search](#randomsearch)
- ir para [Halving Random Serach](#halving)


In [ ]:
R2_mpl      = r2_score(                 y_test0, mpl1.predict(X_test0))
EVS_mpl     = explained_variance_score( y_test0, mpl1.predict(X_test0))
MAE_mpl     = mean_absolute_error(      y_test0, mpl1.predict(X_test0))

R2_lasso    = r2_score(                 y_test0, lasso1.predict(X_test0))
EVS_lasso   = explained_variance_score( y_test0, lasso1.predict(X_test0))
MAE_lasso   = mean_absolute_error(      y_test0, lasso1.predict(X_test0))

R2_bayes    = r2_score(                 y_test0, bayes1.predict(X_test0))
EVS_bayes   = explained_variance_score( y_test0, bayes1.predict(X_test0))
MAE_bayes   = mean_absolute_error(      y_test0, bayes1.predict(X_test0))

R2_elastic  = r2_score(                 y_test0, elastic1.predict(X_test0))
EVS_elastic = explained_variance_score( y_test0, elastic1.predict(X_test0))
MAE_elastic = mean_absolute_error(      y_test0, elastic1.predict(X_test0))

R2_ridge    = r2_score(                 y_test , ridge1.predict(X_test))
EVS_ridge   = explained_variance_score( y_test , ridge1.predict(X_test))
MAE_ridge   = mean_absolute_error(      y_test , ridge1.predict(X_test))

R2_lars     = r2_score(                 y_test ,lars1.predict(X_test))
EVS_lars    = explained_variance_score( y_test ,lars1.predict(X_test))
MAE_lars    = mean_absolute_error(      y_test ,lars1.predict(X_test))

R2_sgd      = r2_score(                 y_test ,sgd1.predict(X_test))
EVS_sgd     = explained_variance_score( y_test ,sgd1.predict(X_test))
MAE_sgd     = mean_absolute_error(      y_test ,sgd1.predict(X_test))

R2_svm      = r2_score(                 y_test0 ,svm1.predict(X_test0))
EVS_svm     = explained_variance_score( y_test0 ,svm1.predict(X_test0))
MAE_svm     = mean_absolute_error(      y_test0 ,svm1.predict(X_test0))

R2_knn      = r2_score(                 y_test0, knn1.predict(X_test0))
EVS_knn     = explained_variance_score( y_test0, knn1.predict(X_test0))
MAE_knn     = mean_absolute_error(      y_test0, knn1.predict(X_test0))

In [ ]:
RSQ = [R2_mpl , R2_lasso , R2_bayes , R2_elastic   , R2_ridge  , R2_lars  , R2_sgd , R2_svm , R2_knn]
EVS = [EVS_mpl, EVS_lasso, EVS_bayes, EVS_elastic  , EVS_ridge , EVS_lars , EVS_sgd, EVS_svm, EVS_knn]
MAE = [MAE_mpl, MAE_lasso, MAE_bayes, MAE_elastic  , MAE_ridge , MAE_lars , MAE_sgd, MAE_svm, MAE_knn]
MOD = ['MLP'  , 'Lasso'  , 'Bayes'  ,'Elastic Net' , 'Ridge'   , 'LARS'   , 'SGD'  ,'SVM Reg', 'KNN']
BHY = [ mpl1.best_params_ ,lasso1.best_params_  ,bayes1.best_params_  ,elastic1.best_params_ ,
        ridge1.best_params_ ,lars1.best_params_ ,sgd1.best_params_  ,svm1.best_params_ , knn1.best_params_]

In [ ]:
resumo_gs = {'Mod' : MOD, 
             'R2'  : RSQ, 
             'EVS' : EVS, 
             'MAE' : MAE,
             'BHY' : BHY}

In [ ]:
pd_resumo_rlv1 = pd.DataFrame(data=resumo_gs)
pd_resumo_rlv1

In [ ]:
pd_resumo_rlv1.to_csv('pd_resumo_rlv1.csv')